In [ ]:
import os, subprocess , IPython
import matplotlib.pyplot as plt
import numpy as np

import essentia.standard as es
#import essentia.streaming

import pylab  #plot, show, figure, imshow
import cv2
import moviepy.editor as mp

import utils.util as util

In [ ]:
''' HELPERS '''
#print(dir(essentia.standard))
print(help(es.AudioLoader))

In [ ]:
#mp4_paths,mp4_labels,aac_paths,aac_labels = util.load_xdv_test(aac_path=util.SERVER_TEST_AUD_ORIG_PATH)
#out1 = util.print_acodec_from_mp4(mp4_paths)
#util.conv_mp4_to_aac(mp4_paths,util.SERVER_TEST_AUD_MONO_PATH,1,False)

In [ ]:
mp4_paths,*_ = util.load_xdv_test(aac_path=util.SERVER_TEST_AUD_MONO_PATH)
out2 = util.print_acodec_from_mp4(mp4_paths,printt=True)

In [ ]:
mp4_paths,mp4_labels,aac_paths,aac_labels = util.load_xdv_test(aac_path=util.SERVER_TEST_AUD_MONO_PATH)
aacpath = aac_paths[44]
mp4path = mp4_paths[44]

#IPython.display.Audio(aacpath)

In [ ]:
#audio = es.MonoLoader(filename=aacpath)()
#from pylab import plot, show, figure, imshow
#pylab.plot(audio[1*44100:2*44100])
#plt.title("This is how the 2nd second of this audio looks like:")
#pylab.show()

#util.plot_mfcc_melbands(audio)

In [1]:
import cv2
import moviepy.editor as mp

In [ ]:
out2 = util.print_acodec_from_mp4([mp4path],printt=True)

video = mp.VideoFileClip(mp4path,)


In [ ]:
video.audio.fps

In [ ]:

audio = mp.AudioFileClip(mp4path,fps=48000)
audio_array = np.array(audio.to_soundarray())

audioaux = audio.subclip(0,2)
audioaux_array = np.array(audioaux.to_soundarray())

print(audio_array.shape , audioaux_array.shape)

audio.close()

In [ ]:
''' writes 10 sec of audio from mp4 cv read synced'''
def get_audio_from_mp4(path,start_t=0.0,end_t=10.0):

    video = cv2.VideoCapture(path)
    fps = video.get(cv2.CAP_PROP_FPS)

    # Extract the audio from the video
    audio = mp.AudioFileClip(path)
    
    while True:
        
        ret, frame = video.read()
        if not ret:break
        video_atual_frame = int(video.get(cv2.CAP_PROP_POS_FRAMES))
        
        start=video.get(cv2.CAP_PROP_POS_MSEC)/1000
        end=(video.get(cv2.CAP_PROP_POS_MSEC)+1000/fps)/1000
        print("1 frame sec start end",start,end)
        
        if video_atual_frame == 240:

            start=start_t
            end=end_t
            print(start,end)
            
            audio_frame = audio.subclip(t_start=start,t_end=end)
            audio_array = np.array(audio_frame.to_soundarray())
            
            nsamples = np.shape(audio_array)[0]
            secs = end-start
            sample_rate = nsamples / secs
            print(audio_array.dtype,np.shape(audio_array) , sample_rate)
            
            # 1 transform stereo to mono
            audio_array_mono = np.mean(audio_array, axis=1)
            es.MonoWriter(filename='audio_mono.wav',sampleRate = int(sample_rate))(audio_array_mono)
            
            # 2 save as stereo
            es.AudioWriter(filename='audio_stereo.wav',sampleRate = int(sample_rate))(audio_array)
            
            break

        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video.release()
    cv2.destroyAllWindows()
    
    
#get_audio_from_mp4(mp4path)